In [4]:
#!/usr/bin/env python
# coding: utf-8

# ## Evaluation Notebook

# !python -m spacy download en_core_web_sm
#pip install spacy

import pandas as pd
from tqdm import tqdm
#import spacy
#from spacy.tokens import DocBin
import re
import os
#from spacy import displacy
from sklearn import metrics
#from sklearn.metrics import plot_confusion_matrix
import random

#from spacy.util import filter_spans

FOODKEEPER_PATH = "datasets/FoodKeeper-Data.xls"
TRAINING_DATA_PATH = "datasets/data.csv"
MODEL_PATH = "output/model-last"
TEST_DATA_PATH = "datasets/test_data.csv"
REAL_TWITTER_DATA_PATH = "datasets/data.csv"
#STARTING_KEYWORD_COUNT = 10
#TRAINING_LOOP_ITERATIONS = 3
#REQUIRED_KEYWORDS = 3

pd.options.mode.chained_assignment = None

In [8]:
food_data = pd.read_excel(FOODKEEPER_PATH, sheet_name = "Product")
all_data = pd.read_csv(TRAINING_DATA_PATH,index_col = False, header = None)
#live_tweets = pd.read_csv(REAL_TWITTER_DATA_PATH, header = None)  
    
#Gathers all the keywords from the FoodKeeper database
def foodKeeperInfo():
    keywords = []
    for word in food_data['Name']: # food_data['Keywords']:
        word = word.replace(" or ", " ") # Replace "or" with a space to separate options
        word = re.sub('[/,]', ', ', word) # Replace commas with ", " to separate phrases
        word = re.sub('\s+', ' ', word) # Replace multiple spaces with a single space
        #word = word.lstrip()
        #word = word.rstrip()
        #word = word.strip()
        #if word.lower() not in keywords: 
            #keywords.append(word.lower())
        words = word.split(", ") # Split the string into a list of words using ", " as the separator
        for w in words:
            w = w.strip() # Remove leading and trailing whitespace from each word
            if w.lower() not in [kw.lower() for kw in keywords]: # # Check if the word already exists in the keywords list (case-insensitive)
                keywords.append(w) # Add the word to the keywords list

    #print("Total foodkeeper food names: " + str(len(keywords)))        
    #for element in sorted(keywords):
        #print(element)
    return keywords


def preProcess(tweet):
    #Converts a tweet to lowercase, replaces anyusername w/ <USERNAME> and URLS with <URL>
    tweet = tweet.lower()
    tweet = re.sub('@[a-zA-z0-9]*', '', tweet)              # <USERNAME>
    tweet = re.sub('http[a-zA-z0-9./:]*', '', tweet)       # <URL>
    tweet = re.sub('[.,-]*', '', tweet)
    tweet = re.sub('&amp;', 'and', tweet)
    return tweet

def unProcess(tweet):
    tweet = re.sub('http[a-zA-z0-9./:]*', '', tweet)       # <URL>
    tweet = re.sub('[.,-]*', '', tweet)
    tweet = re.sub('&amp;', 'and', tweet)
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)
    return tweet


#---------------------------------------------

foodKeeperKeywords = foodKeeperInfo()
print(foodKeeperKeywords)

for i in range(len(all_data[0])):
     all_data[0][i] = unProcess(all_data[0][i])

# TO BE COMPLETED: 
# select tweets which contain one of the foodKeeperKeywords
selected_tweets = []
for tweet in all_data[0]:
    for keyword in foodKeeperKeywords:
        if all(word in tweet for word in keyword):
            selected_tweets.append(tweet)
            break
# then select 250 random tweets of these; this is the test_data
test_data = random.sample(selected_tweets, 250)
test_data_df = pd.DataFrame(test_data, columns=['Random Tweet'])
# save test_data in a "new_test_tweets.csv" file
test_data_df.to_csv('datasets/random_test_tweets.csv', index=False)

# unProcess tweet
foodKeeperKeywords = foodKeeperInfo()
print(foodKeeperKeywords)
# Read the random_test_tweets.csv file
random_test_data_df = pd.read_csv('datasets/random_test_tweets.csv')

# Read the new_test_tweets.csv file
new_test_data_df = pd.read_csv('datasets/new_test_tweets.csv')

# Update the 'Tweet' column in new_test_data_df with the random tweets from random_test_data_df
new_test_data_df['Tweet'] = random_test_data_df['Random Tweet']

# Save the updated new_test_data_df to the new_test_tweets.csv file
new_test_data_df.to_csv('datasets/new_test_tweets.csv', index=False)


['Butter', 'Buttermilk', 'Cheese', 'Coffee creamer', 'Cottage cheese', 'Cream cheese', 'Cream', 'Dips', 'Egg substitutes', 'Eggnog', 'Eggs', 'Egg dishes', 'Kefir', 'Margarine', 'Milk', 'Pudding', 'Sour cream', 'Whipped cream', 'Whipped topping', 'Yogurt', 'Beef', 'Lamb', 'Veal', 'Pork', 'Goat', 'Venison', 'Variety meats', 'Bacon', 'Corned beef', 'Ham', 'Hot dogs', 'Sausage', 'Stuffed', 'raw pork chops', 'Raw kabobs with vegetables', 'Jerky', 'Meat products', 'Retort pouches boxes', 'Chicken', 'Turkey', 'Ground turkey chicken', 'Chicken parts', 'Turkey parts', 'Duckling', 'Goose', 'Pheasant', 'Quail', 'Capon', 'Cornish Hens', 'Giblets', 'raw chicken breasts', 'Turducken', 'Chicken nuggets', 'patties', 'Cooked poultry dishes', 'Fried chicken', 'Poultry pieces', 'Rotisserie chicken', 'Canned chicken', 'Lean fish', 'Fatty fish', 'Caviar', 'Cooked fish', 'Surimi seafood', 'Scallops', 'Shrimp', 'crayfish', 'Squid', 'Shucked clams', 'mussels', 'and oysters', 'Crab meat', 'Crab legs', 'Live cl

In [2]:
# code with spacy

    
def ent_recognize(text):
    doc = nlp(text)
    displacy.render(doc,style = "ent")
    
def predict(tweet):
    doc = nlp(str(tweet))
    if doc.ents:
        displacy.render(doc,style = "ent")

def returnPrediction(tweet):
    nlp = spacy.load(MODEL_PATH)
    doc = nlp(str(tweet))
    if doc.ents:
        return 1
    else:
        return 0
    
def get_predictions():
    predictions = []
    for tweet in test_data['tweet'].tolist():
        predictions.append(returnPrediction(tweet))
    return predictions
    
def eval_model():
    nlp = spacy.load(MODEL_PATH)
    predictions = get_predictions()
    print(metrics.confusion_matrix(y,predictions, labels = [1,0]))
    print(metrics.classification_report(y,predictions, labels = [1,0]))
    
def show_tp():
    counter = 0
    tweets = test_data['tweet'].tolist()
    predictions = get_predictions()
    for i in range(len(y)):
        if predictions[i] == 1 and y[i] == 1:
            print("True positives:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_tn():
    counter = 0
    predictions = get_predictions()
    tweets = test_data['tweet'].tolist()
    for i in range(len(y)):
        if predictions[i] == 0 and y[i] == 0:
            print("True Negative:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_fn():
    predictions = get_predictions()
    tweets = test_data['tweet']
    counter = 0
    for i in range(len(y)):
        if predictions[i] == 0 and y[i] == 1:
            print("False Negative:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_fp():
    predictions = get_predictions()
    tweets = test_data['tweet'].tolist()
    for i in range(len(y)):
        if predictions[i] == 1 and y[i] == 0:
            print("False Positive:")
            doc = nlp(str(tweets[i]))
            if doc.ents:
                displacy.render(doc,style = "ent")

# Evaluation: how well does the existing spacy model identify foods
# Read "new_test_tweets.csv",  

test_data = pd.read_csv(TEST_DATA_PATH)
#test_data = pd.read_csv("datasets/new_test_tweets.csv")

#y = test_data['food'].tolist()
nlp = spacy.load(MODEL_PATH)
#nlp = spacy.blank("en")
print(nlp.pipe_names)
ent_recognize("My rice cakes is tasty")
ent_recognize(" peanut butter and jelly is the classic for me stuff that includes fruit sugars (like apples) are a good choice too imo")


# # Checking for overlapping words
   
#     print(data)
# keywords = foodKeeperInfo()
# testdata = convertToTrainingFormat("My rice cakes is tasty ", keywords)

# # for word in keywords:
# #     for word2 in wordsInFoodkeeper:
# #         if word in word2 and word != word2:
# #             print(word,word2)



# ## Use the function below to check individual sentences
# nlp = spacy.load("en_core_web_sm")
# ent_recognize("my friend is chicken because he is scared")
# print(live_tweets)
# testTweets = live_tweets[5]
# for tweet in testTweets[:500]:
#     if nlp(preProcess(tweet)).ents:
#         ent_recognize(preProcess(tweet))


# ## Use the function below to check model performance on the entire test set
eval_model()

# ## Use the functions below to see TP, TN, FP, FN respectively
# show_tp()
# show_tn()
# show_fp()
#show_fn()

# foodkeeper = foodKeeperInfo()
# print(foodkeeper)
# sortedKeywords =  sorted(keywordRanker, key=keywordRanker.get, reverse=True)

# for i in range(15): #sortedKeywords
#     keywords.append(sortedKeywords[i])
# print(keywords)
# keywords.append("chicken")
# #keywords.append("cream cheese")

# abc = convertToTrainingFormat("I like to eat cream and cheese with chicken test", keywords)
# print(abc)

# # See what keywords are found by the created model
# keywordsFound = []
# nlp = spacy.load(MODEL_PATH)
# print(nlp.pipeline)
# for tweet in live_tweets[5]: #test_data['tweet']:
#     modeledTweet = nlp(preProcess(tweet))
#     for token in modeledTweet.doc.ents:
#         if str(token) in keywordsFound: continue
#         keywordsFound.append(str(token))


# # If the model finds food keywords print the Tweet
# To help visualize which keywords are being found this loop iterates through the Tweets testing the model to see what keywords it finds. If it finds a keyword in the Tweet it will print the Tweet and highlight the keyword

# for tweet in test_data['tweet'][:500]:
#     ents = nlp(preProcess(tweet))
#     #if ents.doc.ents:
#     ent_recognize(preProcess(tweet))

C:\Users\apanangadan\AppData\Local\anaconda3\lib\site-packages\spacy\util.py:887: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


['tok2vec', 'ner']


NameError: name 'y' is not defined